In [1]:
from scqbf.scqbf_instance import *
from scqbf.scqbf_solution import *
from scqbf.scqbf_evaluator import *
from scqbf.scqbf_ga import *
import os
import glob
import time
from concurrent.futures import ProcessPoolExecutor, as_completed
import pandas as pd

In [2]:
instance_paths = [(f"instances/gen{i}/instance{j}.txt", i, j) for i in range(1, 4) for j in range(1, 6)]
instance_paths

[('instances/gen1/instance1.txt', 1, 1),
 ('instances/gen1/instance2.txt', 1, 2),
 ('instances/gen1/instance3.txt', 1, 3),
 ('instances/gen1/instance4.txt', 1, 4),
 ('instances/gen1/instance5.txt', 1, 5),
 ('instances/gen2/instance1.txt', 2, 1),
 ('instances/gen2/instance2.txt', 2, 2),
 ('instances/gen2/instance3.txt', 2, 3),
 ('instances/gen2/instance4.txt', 2, 4),
 ('instances/gen2/instance5.txt', 2, 5),
 ('instances/gen3/instance1.txt', 3, 1),
 ('instances/gen3/instance2.txt', 3, 2),
 ('instances/gen3/instance3.txt', 3, 3),
 ('instances/gen3/instance4.txt', 3, 4),
 ('instances/gen3/instance5.txt', 3, 5)]

In [6]:
def run_single_experiment(args):
    """Helper function to run a single experiment"""
    instance_path, gen, inst, exp_num, config, pop_size, mutation_rate = args
    
    instance = read_max_sc_qbf_instance(instance_path)
    
    time_limit = 600*3
    ga = ScQbfGA(instance, pop_size, mutation_rate, termination_options={'time_limit_secs': time_limit, 'patience': 1000}, ga_strategy=config)
    best_solution = ga.solve()

    evaluator = ScQbfEvaluator(instance)
    result = {
        'gen': gen,
        'inst': inst,
        'n': instance.n,
        'stop_reason': ga.stop_reason,
        'best_objective': evaluator.evaluate_objfun(best_solution),
        'coverage': evaluator.evaluate_coverage(best_solution),
        'time_taken': ga.execution_time,
        'exp_num': exp_num,
        'instance_path': instance_path,
        'selected_elements': best_solution.elements
    }
    
    print(f"Instance {exp_num} completed")
    
    return result

def run_experiment(strategy: GAStrategy, pop_size: int = 100, mutation_rate: float = 1, max_workers: int = None) -> pd.DataFrame:    
    """
    Run experiments in parallel using multiprocessing.
    """
    results = []
    
    experiment_args = [
        (instance_path, gen, inst, exp_num, strategy, pop_size, mutation_rate)
        for exp_num, (instance_path, gen, inst) in enumerate(instance_paths, start=1)
    ]
    
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(run_single_experiment, args) for args in experiment_args]
        
        for future in as_completed(futures):
            try:
                result = future.result()
                results.append(result)
            except Exception as e:
                print(f"Experiment failed with error: {e}")
    
    df = pd.DataFrame(results)
    df = df.sort_values(['gen', 'inst']).reset_index(drop=True)
    
    for _, row in df.iterrows():
        print(f"{row['exp_num']}: {row['inst']}th instance of generation strategy {row['gen']}. Path: {row['instance_path']}")
        print(f"\tBest objective value: {row['best_objective']:.4f}")
        print(f"Selected elements: {row['selected_elements']}")
        print(f"\tCoverage: {row['coverage']:.2%}")
        print(f"\tTime taken (secs): {row['time_taken']:.4f} s")
        print(f"\tStop reason: {row['stop_reason']}")
        print()
    
    return df.drop(columns=['exp_num', 'instance_path', 'selected_elements'])

# Experimentos

In [7]:
P1 = 100
P2 = 50

M1 = 1
M2 = 3

## Experimento 1: PADRÃO
Algoritmo Genético com tamanho de população P1 , taxa de mutação M1 e construção
aleatória da população.

In [5]:
run_experiment(pop_size=P1, mutation_rate=M1, strategy=GAStrategy(population_init='random', mutation_strategy='standard'))

Experiment 4 completed
Experiment 8 completed
Experiment 14 completed
Experiment 15 completed
Experiment 6 completed
Experiment 3 completed
Experiment 12 completed
Experiment 10 completed
Experiment 5 completed
Experiment 1 completed
Experiment 2 completed
Experiment 13 completed
Experiment 11 completed
Experiment 7 completed
Experiment 9 completed
1: 1th instance of generation strategy 1. Path: instances/gen1/instance1.txt
	Best objective value: 4611.4700
Selected elements: [1, 3, 5, 8, 9, 10, 11, 12, 13, 14, 15, 22, 23, 25, 27, 28, 32, 33, 36, 37, 38, 40, 41, 43, 44, 45, 46, 47, 48, 50, 51, 53, 54, 60, 61, 64, 65, 67, 68, 69, 71, 72, 73, 74, 76, 78, 79, 80, 81, 83, 85, 86, 87, 89, 90, 91, 92, 94, 97, 98, 99, 102, 105, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 123, 125, 126, 127, 128, 129, 131, 132, 133, 134, 135, 136, 137, 139, 142, 144, 145, 146, 147, 148, 150, 151, 153, 154, 157, 158, 159, 160, 161, 163, 166, 167, 169, 171, 172, 173, 174, 175, 178, 179, 182, 187, 189, 192, 

,gen,inst,n,stop_reason,best_objective,coverage,time_taken
0,1,1,200,patience_exceeded,4611.47,1.0,256.682109
1,1,2,200,patience_exceeded,4496.87,1.0,261.551133
2,1,3,50,patience_exceeded,645.62,1.0,19.482628
3,1,4,25,patience_exceeded,261.06,1.0,5.386649
4,1,5,100,patience_exceeded,1522.90,1.0,66.221114
5,2,1,50,patience_exceeded,575.87,1.0,16.111046
6,2,2,400,time_limit,14302.26,1.0,1800.028028
7,2,3,25,patience_exceeded,210.16,1.0,6.983432
8,2,4,400,time_limit,13514.06,1.0,1800.177005
9,2,5,100,patience_exceeded,1550.49,1.0,64.330545


## Experimento 2: PADRÃO+POP
Algoritmo Genético PADRÃO mas com tamanho de população P2 .

In [ ]:
run_experiment(pop_size=P2, mutation_rate=M1, strategy=GAStrategy(population_init='random', mutation_strategy='standard'))

## Experimento 3: PADRÃO+MUT
Algoritmo Genético PADRÃO mas com taxa de mutação M2 .

In [ ]:
run_experiment(pop_size=P1, mutation_rate=M2, strategy=GAStrategy(population_init='random', mutation_strategy='standard'))

## Experimento 4: PADRÃO+EVOL1
Algoritmo Genético PADRÃO mas com estratégia evolutiva alternativa 1 (Latin Hypercube)

In [ ]:
run_experiment(pop_size=P1, mutation_rate=M1, strategy=GAStrategy(population_init='latin_hypercube', mutation_strategy='standard'))

## Experimento 5: PADRÃO+EVOL2
Algoritmo Genético PADRÃO mas com estratégia evolutiva alternativa 2 (adaptive mutation)

In [ ]:
run_experiment(pop_size=P1, mutation_rate=M1, strategy=GAStrategy(population_init='random', mutation_strategy='adaptive'))